In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import os
from datetime import datetime, UTC
import analytiq_data as ad
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
import docrouter_app.payments

In [4]:
ad.common.setup()
await docrouter_app.payments.init_payments_env()

In [5]:
#await docrouter_app.payments.delete_all_payments_customers(dryrun=False)

In [5]:
org_id = "6795345439604beca2b2808d"

In [6]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

2025-07-04 09:41:42,630 - docrouter_app.payments - INFO - Sync stripe customer for org_id: 6795345439604beca2b2808d user_id: 679533ee39604beca2b2803a email: iubica2@yahoo.com name: System Administrator org_name: test
2025-07-04 09:41:42,639 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%276795345439604beca2b2808d%27
2025-07-04 09:41:42,904 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%276795345439604beca2b2808d%27 response_code=200
2025-07-04 09:41:42,905 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/prices?active=True&expand[0]=data.product
2025-07-04 09:41:43,307 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/prices?active=True&expand[0]=data.product response_code=200
2025-07-04 09:41:43,310 - docrouter_app.payments - INFO - Dynamic tie

{'_id': ObjectId('6867d4da0447e3044e44aa17'),
 'org_id': '679c9a914cfaaaa3640811ed',
 'user_id': '679533ee39604beca2b2803a',
 'stripe_customer_id': 'cus_SbGx22QE979U2r',
 'user_name': 'System Administrator',
 'user_email': 'iubica2@yahoo.com',
 'has_payment_method': False,
 'created_at': datetime.datetime(2025, 7, 4, 13, 19, 22, 813000),
 'updated_at': datetime.datetime(2025, 7, 4, 13, 34, 29, 973000),
 'spu_credits': 500,
 'spu_credits_used': 0,
 'name': 'System Administrator'}

In [ ]:
stripe_customer = await docrouter_app.payments.get_payments_customer(org_id=org_id)
stripe_customer

In [ ]:
stripe_customer.id

In [ ]:
subscription = await docrouter_app.payments.get_subscription(stripe_customer.id)
subscription

In [7]:
await docrouter_app.payments.get_stripe_usage(org_id=org_id)

2025-07-04 09:41:47,889 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%276795345439604beca2b2808d%27
2025-07-04 09:41:48,069 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%276795345439604beca2b2808d%27 response_code=200
2025-07-04 09:41:48,072 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SbGxOTbDNTq6wa
2025-07-04 09:41:48,258 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/subscriptions?customer=cus_SbGxOTbDNTq6wa response_code=200


2025-07-04 09:41:48,260 - docrouter_app.payments - INFO - Current period start: 2025-07-01T10:51:10
2025-07-04 09:41:48,260 - docrouter_app.payments - INFO - Current period end: 2025-08-01T10:51:10


{'total_usage': 400,
 'metered_usage': 400,
 'period_start': 1751381470,
 'period_end': 1754059870,
 'subscription_type': 'team',
 'usage_unit': 'spu'}

In [ ]:
await docrouter_app.payments.process_all_billing()

In [ ]:
await docrouter_app.payments.process_org_billing(org_id=org_id)

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [ ]:
await docrouter_app.payments.save_usage_record(org_id=org_id, spus=100, operation="test")

2025-07-04 09:41:53,482 - docrouter_app.payments - INFO - handle_usage_record() called with org_id: 6795345439604beca2b2808d, spus: 100, operation: test, source: backend


{'org_id': '6795345439604beca2b2808d',
 'spus': 100,
 'operation': 'test',
 'source': 'backend',
 'timestamp': datetime.datetime(2025, 7, 4, 13, 41, 53, 483312),
 'reported_to_stripe': False,
 '_id': ObjectId('6867da211e41d7206bae378c')}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [38]:
org_id = "6859696f6ebcee0b712b9ac6"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
await docrouter_app.payments.delete_payments_customer(org_id=org_id, force=False)

In [ ]:
await docrouter_app.payments.get_tier_config()